In [99]:
import pandas as pd
df=pd.read_csv('abalone.data')
# num=df.shape[1]
# print(num)
# for i in range(num):
#     df=df[df.iloc[:,i]!='?']
target=df.iloc[:,-1]

df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9,16,18,20]],axis=1,inplace=True)
df.columns=['Age','Gender','Ethnicity','Jaundice(YES/NO)','Autism in family?','Country','Score','Representative']
df

,Age,Gender,Ethnicity,Jaundice(YES/NO),Autism in family?,Country,Score,Representative
0,24,m,Latino,no,yes,Brazil,5,Self
1,27,m,Latino,yes,yes,Spain,8,Parent
2,35,f,White-European,no,yes,'United States',6,Self
3,40,f,?,no,no,Egypt,2,?
4,36,m,Others,yes,no,'United States',9,Self
...,...,...,...,...,...,...,...,...
698,25,f,White-European,no,no,Russia,7,Self
699,34,m,Hispanic,no,no,Mexico,3,Parent
700,24,f,?,no,no,Russia,7,?
701,35,m,'South Asian',no,no,Pakistan,6,Self


In [100]:
def fit(x_train,y_train):
    result={}
    classes=set(y_train)
    for i in classes:
        result[i]={}
        result[i]['tot_sum']=(y_train==i).sum()
        x_new=x_train[y_train==i]
        num=x_new.shape[1]
        for j in range(num):
            result[i][j]={}
            values=set(x_train.iloc[:,j])
            for z in values:
                if(z=='?'):
                    continue
                result[i][j][z]=(x_new.iloc[:,j]==z).sum()
                
    return result            

In [102]:
def predict(result,x_test,x):
    y_pred=[]
    num=x_test.shape[0]
    for i in range(num):
        max_prob=0
        ans=0
        for j in result:
            temp=result[j]['tot_sum']/x.shape[0]
            size=x_test.shape[1]
            for z in range(size):
                if(x_test.iloc[i,z]=='?'):
                    continue
                if(x_test.iloc[i,z] not in result[j][z].keys()):
                    result[j][z][x_test.iloc[i,z]]=0
#                 print(result[j][z][x_test.iloc[i,z]]+1,(result[j]['tot_sum'])+len(result[j])-1)    
                temp=temp*(result[j][z][x_test.iloc[i,z]]+1)/((result[j]['tot_sum'])+len(result[j])-1)
#             print(j,temp)    
            if(temp>max_prob):
                max_prob=temp
                ans=j
#         print(ans)        
        y_pred.append(ans)
   
    return y_pred                 

In [114]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df,target)

In [115]:
result=fit(x_train,y_train)
# print(result)
y_pred=predict(result,x_test,x_train)
y_pred
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))


[[131   0]
 [  1  44]]
              precision    recall  f1-score   support

          NO       0.99      1.00      1.00       131
         YES       1.00      0.98      0.99        45

    accuracy                           0.99       176
   macro avg       1.00      0.99      0.99       176
weighted avg       0.99      0.99      0.99       176

